In [16]:
import findspark
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder. \
    appName("pyspark-1"). \
    getOrCreate()

In [4]:
spark.version

'2.4.5'

In [7]:
df = spark.read.csv("/dataset/pyspark-df-overview/census_income.csv.gz", header=True)
df.printSchema()

root
 |-- age: string (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: string (nullable = true)
 |-- education: string (nullable = true)
 |-- education.num: string (nullable = true)
 |-- marital.status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital.gain: string (nullable = true)
 |-- capital.loss: string (nullable = true)
 |-- hours.per.week: string (nullable = true)
 |-- native.country: string (nullable = true)
 |-- income: string (nullable = true)



In [32]:
x = df.select("age","fnlwgt").withColumn("age_group", F.when(F.col("age")<=30, F.lit("youth-age")).otherwise(F.when(((F.col("age")>30) & (F.col("age")<=50)), F.lit("mid-age")).otherwise(F.lit("old-age"))))

In [33]:
def sample_function_get_max_fnlwgt(df: DataFrame) -> DataFrame:
    df_age_grp = df.select("fnlwgt", "age").withColumn("age_group", F.when(F.col("age")<=30, F.lit("youth-age")).otherwise(F.when(((F.col("age")>30) & (F.col("age")<=50)), F.lit("mid-age")).otherwise(F.lit("old-age"))))
    output = df_age_grp.groupBy(F.col("age_group")).agg(F.max(F.col("fnlwgt")).alias("max_fnlwgt"))
    return output

DataFrame[age: string, workclass: string, fnlwgt: string, education: string, education.num: string, marital.status: string, occupation: string, relationship: string, race: string, sex: string, capital.gain: string, capital.loss: string, hours.per.week: string, native.country: string, income: string, age_group: string]

In [35]:
xx = sample_function_get_max_fnlwgt(x)

In [36]:
xx.show()

+---------+----------+
|age_group|max_fnlwgt|
+---------+----------+
|youth-age|     99970|
|  mid-age|     99971|
|  old-age|     99987|
+---------+----------+

